In [1]:
#Theese are the dependencies for our VMF program.
###
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
###

import VMF




GPUs are used!
Classes loaded
0


/uio/hume/student-u44/lmsunde/projects/BNN/AliaksandrFolder/VMF.py:279: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v0 = torch.cat([v0, torch.tensor(w0[det >= 0]).to(DEVICE)])
/opt/uio/modules/rhel8/easybuild/software/Miniconda3/lmsunde/envs/BNN2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
tensor(51382.0586, device='cuda:1', grad_fn=<AddBackward0>)
tensor(166.6270, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(8.5852e-05, device='cuda:1', grad_fn=<MeanBackward0>)
2
tensor(51373.0234, device='cuda:1', grad_fn=<AddBackward0>)
tensor(156.5161, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(8.8558e-05, device='cuda:1', grad_fn=<MeanBackward0>)
3
tensor(51373.0117, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.1649, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(2.1525e-05, device='cuda:1', grad_fn=<MeanBackward0>)
4
tensor(51354.9180, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.2581, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(5.5383e-06, device='cuda:1', grad_fn=<MeanBackward0>)
5
tensor(51348.1758, device='cuda:1', grad_fn=<AddBackward0>)
tensor(162.2833, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-2.8050e-06, device='cuda:1', grad_fn=<MeanBackward0>)
6
tensor(51337.4805, device='cuda:1', grad_fn=<AddBackward0>)
tensor(161.4

46
tensor(50876.3320, device='cuda:1', grad_fn=<AddBackward0>)
tensor(119.3678, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0057, device='cuda:1', grad_fn=<MeanBackward0>)
47
tensor(50856.7266, device='cuda:1', grad_fn=<AddBackward0>)
tensor(121.4119, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0058, device='cuda:1', grad_fn=<MeanBackward0>)
48
tensor(50855.6758, device='cuda:1', grad_fn=<AddBackward0>)
tensor(118.5983, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0060, device='cuda:1', grad_fn=<MeanBackward0>)
49
tensor(50854.0703, device='cuda:1', grad_fn=<AddBackward0>)
tensor(123.4238, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0061, device='cuda:1', grad_fn=<MeanBackward0>)
50
tensor(50806.8477, device='cuda:1', grad_fn=<AddBackward0>)
tensor(107.3720, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0062, device='cuda:1', grad_fn=<MeanBackward0>)
51
tensor(50808.1484, device='cuda:1', grad_fn=<AddBackward0>)
tensor(120.6393, devic

91
tensor(50368.1875, device='cuda:1', grad_fn=<AddBackward0>)
tensor(92.3183, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0105, device='cuda:1', grad_fn=<MeanBackward0>)
92
tensor(50381.9453, device='cuda:1', grad_fn=<AddBackward0>)
tensor(99.9593, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0106, device='cuda:1', grad_fn=<MeanBackward0>)
93
tensor(50356.1953, device='cuda:1', grad_fn=<AddBackward0>)
tensor(94.6393, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0107, device='cuda:1', grad_fn=<MeanBackward0>)
94
tensor(50342.4766, device='cuda:1', grad_fn=<AddBackward0>)
tensor(96.0689, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0108, device='cuda:1', grad_fn=<MeanBackward0>)
95
tensor(50315.1094, device='cuda:1', grad_fn=<AddBackward0>)
tensor(87.0866, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0109, device='cuda:1', grad_fn=<MeanBackward0>)
96
tensor(50308.9258, device='cuda:1', grad_fn=<AddBackward0>)
tensor(87.1057, device='cud

136
tensor(49891.6055, device='cuda:1', grad_fn=<AddBackward0>)
tensor(87.7965, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0138, device='cuda:1', grad_fn=<MeanBackward0>)
137
tensor(49873.3633, device='cuda:1', grad_fn=<AddBackward0>)
tensor(83.7328, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0139, device='cuda:1', grad_fn=<MeanBackward0>)
138
tensor(49863.3594, device='cuda:1', grad_fn=<AddBackward0>)
tensor(85.4622, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0140, device='cuda:1', grad_fn=<MeanBackward0>)
139
tensor(49846.3867, device='cuda:1', grad_fn=<AddBackward0>)
tensor(83.8477, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0140, device='cuda:1', grad_fn=<MeanBackward0>)
140
tensor(49841.2109, device='cuda:1', grad_fn=<AddBackward0>)
tensor(86.1955, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0140, device='cuda:1', grad_fn=<MeanBackward0>)
141
tensor(49846.8789, device='cuda:1', grad_fn=<AddBackward0>)
tensor(88.3962, devic

181
tensor(49419.6445, device='cuda:1', grad_fn=<AddBackward0>)
tensor(87.2795, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0153, device='cuda:1', grad_fn=<MeanBackward0>)
182
tensor(49394.8477, device='cuda:1', grad_fn=<AddBackward0>)
tensor(79.6318, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0153, device='cuda:1', grad_fn=<MeanBackward0>)
183
tensor(49380.3438, device='cuda:1', grad_fn=<AddBackward0>)
tensor(82.5515, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0154, device='cuda:1', grad_fn=<MeanBackward0>)
184
tensor(49393.0352, device='cuda:1', grad_fn=<AddBackward0>)
tensor(86.1629, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0154, device='cuda:1', grad_fn=<MeanBackward0>)
185
tensor(49360.5117, device='cuda:1', grad_fn=<AddBackward0>)
tensor(83.7876, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0155, device='cuda:1', grad_fn=<MeanBackward0>)
186
tensor(49344.6992, device='cuda:1', grad_fn=<AddBackward0>)
tensor(77.0241, devic

226
tensor(48953.6953, device='cuda:1', grad_fn=<AddBackward0>)
tensor(74.1071, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0157, device='cuda:1', grad_fn=<MeanBackward0>)
227
tensor(48933.0234, device='cuda:1', grad_fn=<AddBackward0>)
tensor(74.1348, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0158, device='cuda:1', grad_fn=<MeanBackward0>)
228
tensor(48903.8945, device='cuda:1', grad_fn=<AddBackward0>)
tensor(72.1158, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0158, device='cuda:1', grad_fn=<MeanBackward0>)
229
tensor(48901.5938, device='cuda:1', grad_fn=<AddBackward0>)
tensor(75.4586, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0158, device='cuda:1', grad_fn=<MeanBackward0>)
230
tensor(48900.7617, device='cuda:1', grad_fn=<AddBackward0>)
tensor(71.9722, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(-0.0158, device='cuda:1', grad_fn=<MeanBackward0>)
231
tensor(48875.1406, device='cuda:1', grad_fn=<AddBackward0>)
tensor(68.9379, devic

In [2]:
#epochs = 225
#trtimes  = np.zeros(epochs)
# make inference on 10 networks
#for i in range(0, 1):
#    print(i)
#    torch.manual_seed(i)
#    net = VMF.BayesianNetwork().to(VMF.DEVICE)
#    optimizer = optim.Adam(net.parameters(), lr=0.0001)
#    for epoch in range(epochs):
#
#        trtimes[epoch] = VMF.train(net, optimizer, epoch, i)
#        print(net.l1.weight_mu.mean())
#
#    res = VMF.test_ensemble()
#
#    np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
